## ASTR 597A Homework 5
Eric Bellm

Due Feb 7, 2023

Your name:
    
Your collaborators:

In this homework we will attempt to "replicate" a classic SDSS paper ([Baldry et al. 2004](https://ui.adsabs.harvard.edu/abs/2004ApJ...600..681B/abstract)) with simulated Rubin data. Plotting the absolute r-band magnitudes of low-redshift galaxies vs. their (u-r) colors shows clearly the red sequence, blue clump, and green valley populations.  We will investigate if the DP0.2 galaxies show the same distribution, explore the implications since we won't have (many) spectroscopic redshifts, and look at possible evolution with redshift...

Portions of this work were developed based on [this notebook](https://github.com/rubin-dp0/delegate-contributions-dp02/tree/main/photoz/CMNN_Estimator) by Melissa Graham.

We will need to deal with "k-corrections" in order to convert our observer frame photometry to the rest frame.  See [Hogg et al. 2002](https://ui.adsabs.harvard.edu/abs/2002astro.ph.10394H/abstract) for a pedagogical overview.

The python package `kcorrect` by Mike Blanton provides our easist way to deal with this.  The package documentation is [here](https://kcorrect.readthedocs.io/), the source code [here](https://github.com/blanton144/kcorrect), and the paper describing the algorithm is [here](https://ui.adsabs.harvard.edu/abs/2007AJ....133..734B/abstract).

In [ ]:
# As of Jan. 29, 2023, pypi version throws an error: install from github directly
!pip install --user git+https://github.com/blanton144/kcorrect.git

## Exercise 1: Low-redshift galaxy CMD with LSST

Plot the absolute r magnitude vs. rest-frame u-r color for 50,000 galaxies with z < 0.08.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import numpy.ma as ma

In [ ]:
from lsst.rsp import get_tap_service, retrieve_query
service = get_tap_service()

Since we're going to need more columns than you might expect in this homework and the query is slow (~20 minutes), I'm going to provide the SQL query:

In [ ]:
query = "SELECT mt.id_truth_type AS mt_id_truth_type, "\
        "mt.match_objectId AS mt_match_objectId, "\
        "ts.truth_type AS ts_truth_type, "\
        "ts.redshift AS ts_redshift, "\
        "scisql_nanojanskyToAbMag(ts.flux_u) AS ts_mag_u, "\
        "scisql_nanojanskyToAbMag(ts.flux_g) AS ts_mag_g, "\
        "scisql_nanojanskyToAbMag(ts.flux_r) AS ts_mag_r, "\
        "scisql_nanojanskyToAbMag(ts.flux_i) AS ts_mag_i, "\
        "scisql_nanojanskyToAbMag(ts.flux_z) AS ts_mag_z, "\
        "scisql_nanojanskyToAbMag(ts.flux_y) AS ts_mag_y, "\
        "scisql_nanojanskyToAbMag(obj.u_cModelFlux) AS obj_cModelMag_u, "\
        "scisql_nanojanskyToAbMag(obj.g_cModelFlux) AS obj_cModelMag_g, "\
        "scisql_nanojanskyToAbMag(obj.r_cModelFlux) AS obj_cModelMag_r, "\
        "scisql_nanojanskyToAbMag(obj.i_cModelFlux) AS obj_cModelMag_i, "\
        "scisql_nanojanskyToAbMag(obj.z_cModelFlux) AS obj_cModelMag_z, "\
        "scisql_nanojanskyToAbMag(obj.y_cModelFlux) AS obj_cModelMag_y, "\
        "scisql_nanojanskyToAbMagSigma(obj.u_cModelFlux,obj.u_cModelFluxErr) AS obj_cModelMagErr_u, "\
        "scisql_nanojanskyToAbMagSigma(obj.g_cModelFlux,obj.g_cModelFluxErr) AS obj_cModelMagErr_g, "\
        "scisql_nanojanskyToAbMagSigma(obj.r_cModelFlux,obj.r_cModelFluxErr) AS obj_cModelMagErr_r, "\
        "scisql_nanojanskyToAbMagSigma(obj.i_cModelFlux,obj.i_cModelFluxErr) AS obj_cModelMagErr_i, "\
        "scisql_nanojanskyToAbMagSigma(obj.z_cModelFlux,obj.z_cModelFluxErr) AS obj_cModelMagErr_z, "\
        "scisql_nanojanskyToAbMagSigma(obj.y_cModelFlux,obj.y_cModelFluxErr) AS obj_cModelMagErr_y "\
        "FROM dp02_dc2_catalogs.MatchesTruth AS mt "\
        "JOIN dp02_dc2_catalogs.TruthSummary AS ts ON mt.id_truth_type = ts.id_truth_type "\
        "JOIN dp02_dc2_catalogs.Object AS obj ON mt.match_objectId = obj.objectId "\
        "WHERE ts.truth_type = 1 "\ 
        "AND ts.redshift < 0.08 "\
        "LIMIT 50000"
print(query)

In [ ]:
%%time
job = service.submit_job(query)
print('Job URL is', job.url)
print('Job phase is', job.phase)
job.run()
job.wait(phases=['COMPLETED', 'ERROR'])
print('Job phase is', job.phase)

In [ ]:
#job.raise_if_error()

In [ ]:
df = job.fetch_result().to_table().to_pandas()

In [ ]:
# since the query is slow let's save the results
df.to_parquet('lowz_query_results.parquet')

In [ ]:
# reread from file if needed
# df = pd.read_parquet('lowz_query_results.parquet')

We now have the true redshifts and both true and "observed" magnitudes in all bands for sources we know are galaxies (`ts.truth_type == 1`; this is a bit of a cheat!).  To get the absolute magnitude we need the distance modulus and the k-correction (see the references above, or [here](https://kcorrect.readthedocs.io/en/5.0.0b/basics.html#) for a briefer overview).  (The rest-frame color requires the k-corrections as well.)  To use `kcorrect` we need to make some unit conversions.

`kcorrect` doesn't have LSST bands in it at present, so we will use [DECam's filterset](https://noirlab.edu/science/programs/ctio/filters/Dark-Energy-Camera).  Take a moment to compare them to LSST's filters--are there significant differences?  

In [ ]:
import kcorrect.kcorrect

responses = ['decam_u', 'decam_g', 'decam_r', 'decam_i', 'decam_z', 'decam_Y']
kc = kcorrect.kcorrect.Kcorrect(responses=responses)

`kcorrect` wants inputs in units of maggies and their inverse variances, as numpy arrays with dimensions (objects x filters).  For convenience here is that conversion:

In [ ]:
maggies = df[['obj_cModelMag_u', 'obj_cModelMag_g', 'obj_cModelMag_r',
       'obj_cModelMag_i', 'obj_cModelMag_z', 'obj_cModelMag_y']].apply(
    lambda x: 10.0 ** (-0.4 * x)).values
maggies

In [ ]:
ivars = df[['obj_cModelMagErr_u', 'obj_cModelMagErr_g', 'obj_cModelMagErr_r',
       'obj_cModelMagErr_i', 'obj_cModelMagErr_z', 'obj_cModelMagErr_y']].apply(
    lambda x: 1./ (0.4 * np.log(10) * x) ** 2.).values / (maggies**2.)
ivars

Now use `kcorrect` to compute the rest-frame absolute magnitude in r-band as well as the rest frame u-r color, and plot them.  Note that you'll need to exclude some rows with `NaN`s.

Compare your result qualitatively to Figure 1 of Baldry et al. 2004.

## Exercise 2: Removing a cheat

We made two cheats above: we used the truth table to tell us which galaxies we have as well as their redshifts.  In SDSS those redshifts were obtained through spectroscopy, but for Rubin most redshifts will have to be obtained photometrically.

Using the data from our query above, follow [this notebook](https://github.com/rubin-dp0/delegate-contributions-dp02/tree/main/photoz/CMNN_Estimator) by Melissa Graham to compute a very simple photo-z for these galaxies.  Plot the true redshift vs estimated photo-z, and then re-plot the CMD from Exercise 1 using photo-z rather than the true spectroscopic redshift.

## Exercise 3 (Optional): Hi-Z

Repeat exercise 1 for a higher redshift range (z=1.5-2?) and see how the CMD changes.